In [1]:
from tefas import Crawler
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# Define date range for data retrieval
start_date = datetime(2018, 1, 1)
end_date = datetime(2024, 9, 1)
date_increment = timedelta(days=30)

# Initialize the TEFAS Crawler
tefas_crawler = Crawler()

# List of fund codes and columns
fund_codes = ["KPC", "HKH", "KLU", "KZL", "CPU", "KUT"]
columns = ["date", "code", "title", "price", "market_cap", "number_of_investors", "number_of_shares"]

In [4]:
# Data Retrieval Loop
all_funds_data_list = []

for fund in fund_codes:
    current_date = start_date
    while current_date <= end_date:
        try:
            # Fetch data for each fund and date range
            fund_data = tefas_crawler.fetch(
                start=current_date.strftime('%Y-%m-%d'),
                end=(current_date + date_increment).strftime('%Y-%m-%d'),
                name=fund
            )
            # Add the fund code to the data
            fund_data['code'] = fund
            # Select only the columns we need
            fund_data = fund_data[['date', 'code', 'title', 'price', 'market_cap', 'number_of_investors', 'number_of_shares']]
            # Drop rows where all elements are NaN
            fund_data = fund_data.dropna(how='all')
            # Check if fund_data is not empty after dropping NaNs
            if not fund_data.empty:
                all_funds_data_list.append(fund_data)
        except Exception as e:
            print(f"Error fetching data for {fund} from {current_date}: {e}")
        # Move to the next date range
        current_date += date_increment

# Combine all the data into a single DataFrame
all_funds_data = pd.concat(all_funds_data_list, ignore_index=True)

In [5]:
# Sort the DataFrame by 'date' column
all_funds_data = all_funds_data.sort_values(by='date')
all_funds_data = all_funds_data.reset_index(drop=True)

In [6]:
all_funds_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4713 entries, 0 to 4712
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   date                 4713 non-null   object 
 1   code                 4713 non-null   object 
 2   title                4713 non-null   object 
 3   price                4713 non-null   float64
 4   market_cap           4713 non-null   float64
 5   number_of_investors  4713 non-null   float64
 6   number_of_shares     4713 non-null   float64
dtypes: float64(4), object(3)
memory usage: 257.9+ KB


In [7]:
all_funds_data

,date,code,title,price,market_cap,number_of_investors,number_of_shares
0,2019-09-23,KZL,KUVEYT TÜRK ASSET MANAGEMENT GOLD PARTICIPATIO...,1.175174,8.308115e+06,298.0,7.069690e+06
1,2019-09-24,KZL,KUVEYT TÜRK ASSET MANAGEMENT GOLD PARTICIPATIO...,1.193278,8.495458e+06,295.0,7.119429e+06
2,2019-09-25,KZL,KUVEYT TÜRK ASSET MANAGEMENT GOLD PARTICIPATIO...,1.185895,8.384649e+06,288.0,7.070313e+06
3,2019-09-26,KZL,KUVEYT TÜRK ASSET MANAGEMENT GOLD PARTICIPATIO...,1.194965,8.071311e+06,287.0,6.754434e+06
4,2019-09-27,KZL,KUVEYT TÜRK ASSET MANAGEMENT GOLD PARTICIPATIO...,1.181067,7.889600e+06,285.0,6.680062e+06
...,...,...,...,...,...,...,...
4708,2024-09-02,KLU,KUVEYT TÜRK PORTFÖY PARA PİYASASI KATILIM (TL)...,2.282647,4.102847e+10,56230.0,1.797407e+10
4709,2024-09-02,HKH,HEDEF PORTFÖY KATILIM HİSSE SENEDİ (TL) FONU (...,6.045513,2.587774e+09,34374.0,4.280487e+08
4710,2024-09-02,CPU,AKTİF PORTFÖY TEKNOLOJİ KATILIM FONU,1.486352,7.326663e+08,21153.0,4.929293e+08
4711,2024-09-02,KUT,KUVEYT TÜRK PORTFÖY KIYMETLİ MADENLER KATILIM ...,4.613837,1.118915e+09,13860.0,2.425128e+08


In [8]:
all_funds_data.to_csv('tefas_funds_data.csv', index=False, encoding='utf-8')